In [1]:
from scipy import constants,misc as cs,misc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lmfit.models import GaussianModel,LinearModel
from glob import glob
from lmfit import models

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
%matplotlib qt

In [3]:
cd ~/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/

/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite


In [4]:
file_list = []
liste_reich = []
for i in glob('/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/*.txt'):
    file_list.append(i)

    
file_list.sort()   
    
for i in file_list:
    liste_reich.append(np.array(pd.read_csv(i,skiprows=0,delim_whitespace=2)).T)

file_list

['/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/Reichweite12mbar.txt',
 '/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/Reichweite190mbar.txt',
 '/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/Reichweite200mbar.txt',
 '/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/Reichweite300mbar.txt',
 '/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/Reichweite400mbar.txt',
 '/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/Reichweite450mbar.txt',
 '/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/Reichweite475mbar.txt',
 '/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/Reichweite500mbar.txt',
 '/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/Reichweite510mbar.txt',
 '/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/Reichweite/Reichweite525mbar.txt',
 '/home/frederik/Dokumente/FP/Rutherford/

In [5]:
druck = np.array([12,100,200,300,400,450,475,500,510,525,540,550,560,575,600,700])

In [6]:
def poisson(x,B,A=2):
    return B*((A**x/misc.factorial(x))*np.e**(-A))

In [7]:
mod_pos = models.Model(poisson)

In [28]:
err =[]
reich_fit_reports = []
reich_fit_pars = []
for i in liste_reich:
    if i[0].size < 1:
        continue
    err_temp = None
    err_temp = np.sqrt(i[1])
    err_temp = np.where(err_temp == 0, 1, err_temp)
    err.append(np.where(np.sqrt(i[1]) == 0, 1, np.sqrt(i[1])))
    pars = mod_pos.make_params()
    pars['B'].set(vary=False)
    fit = mod_pos.fit(i[1],
                  pars,
                  A= np.sum(i[0]*i[1]/180),
                  B=np.sum(i[1]),
                  weights=1/err_temp,
                  x=i[0])
    reich_fit_reports.append(fit.fit_report())
    reich_fit_pars.append(np.array([fit.best_values,fit.chisqr]))
    fit.plot()
    plt.show()

In [9]:
counts = []
for i in reich_fit_pars:
    print(i[0]["A"])
    counts.append(i[0]["A"])

52.1842854011
53.5525444782
53.7023158872
53.9305238687
51.356502194
31.8024267261
16.2677491547
4.71723264871
2.21370160198
0.546468530406
0.197558073689
0.0486455630319
0.0169491112976
0.00558659177403
1.49011611939e-08
1.49011611939e-08


In [10]:
plt.errorbar(druck,counts,np.sqrt(counts),fmt='. g')
plt.show()

In [11]:
def logistisch(druck,A,B,C,D):
    return B+(A-B)/(1+(druck/D)**C)

In [12]:
mod_logistic = models.Model(logistisch)
pars = mod_logistic.make_params(A=52,B=-1,C=25,D=450)

In [13]:
out = mod_logistic.fit(counts,pars,druck=druck)

In [14]:
out.plot()
plt.show()
print(out.fit_report())

[[Model]]
    Model(logistisch)
[[Fit Statistics]]
    # function evals   = 31
    # data points      = 16
    # variables        = 4
    chi-square         = 5.368
    reduced chi-square = 0.447
[[Variables]]
    B:  -0.50258110 +/- 0.293587 (58.42%) (init=-1)
    C:   24.7873000 +/- 1.057070 (4.26%) (init= 25)
    A:   53.2844020 +/- 0.312566 (0.59%) (init= 52)
    D:   458.216791 +/- 0.808596 (0.18%) (init= 450)
[[Correlations]] (unreported correlations are <  0.100)
    C(B, C)                      =  0.546 
    C(A, D)                      = -0.336 
    C(B, D)                      = -0.266 
    C(C, A)                      = -0.233 
    C(C, D)                      =  0.167 


In [15]:
xs = np.arange(0,700,0.1)

In [17]:
plt.plot(xs,logistisch(xs,
                          out.best_values['A'],
                          out.best_values['B'],
                          out.best_values['C'],
                          out.best_values['D']),'-r',label="Fit")
plt.errorbar(druck,counts,np.sqrt(counts)/5,fmt='. g',label="Messdaten")
plt.ylabel('Counts', fontsize = 42)
plt.xlabel(r'Druck / mbar' , fontsize = 42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best', fontsize = 42)
plt.show()

In [18]:
out.best_values['D']**out.best_values['C']*np.sqrt(-1*out.best_values['A']/out.best_values['B'])

9.4072442160480325e+66

In [19]:
458* np.sqrt(53/0.5)**(1/24)

504.73026872318667

In [20]:
((5/1013.25)*504)

2.4870466321243523

In [21]:
1/1013.25*np.sqrt((0.5*504)**2 + (34*5)**2)

0.30000519370062423

In [22]:
((5/1013.25)*504)*5.4/4.5

2.9844559585492227

In [23]:
1/1013.25*np.sqrt((0.5*504)**2 + (34*5)**2)*5.4/4.5

0.36000623244074909

In [24]:
100*abs(1-(2.98/2.96))

0.6756756756756799